In [ ]:
import os
import anndata as ad
from bmark.utils.dataset import get_kfold_ind

### Load taxonomy/dataset

In [ ]:
os.chdir("/allen/programs/celltypes/workgroups/hct/cellTaxonomy/adult-human-brain_v1/additional_files/processed/CSR")
taxonomy = ad.read_h5ad("human_whole_brain_subcluster_centroid_subsampled.h5ad", backed="r")

#### Ensure subclusters have n >= 50 cells to support 10 fold cross-valiationd

In [ ]:
## taxonomy must have n > 50 cells per subcluster
counts = taxonomy.obs["subcluster_id"].value_counts() > 50
taxonomy = taxonomy[taxonomy.obs["subcluster_id"].isin(counts[counts].index.tolist())]

### Configure k-fold validation
Derive 10-fold train, test splits and store in the .uns of the anndata object.

In [ ]:
## Define where to store benchmarking information in the anndata object
taxonomy.uns["benchmark"] = {"k_fold": {}}

## Precompute all 10 train and validation splits of the data
for fold in range(10):
    train_ind, validation_ind = get_kfold_ind(obs=taxonomy.obs, stratify_by='subcluster_id', fold=fold, n_folds=10)
    taxonomy.uns["benchmark"]["k_fold"]["fold_" + str(fold+1)] = {"train_ind": train_ind, "val_ind": validation_ind}

### Save the benchmark

In [ ]:
taxonomy.write_h5ad("Siletti_Human_WholeBrain_benchmark.h5ad")